<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/master/doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preview of CNN based on bee_classification
-------------------

## Spis treści
1. [Wstęp](#1)
    1. [Cel projektu](#1.1)
    2. [Czym jest CNN?](#1.2)
    3. [Z czego składa się bee_dataset?](#1.3)
2. [Model](#2)
    1. [Specyfika modelu](#2.1)
    2. [Część eksperymentalna](#2.2)
        1. [Architektura](#2.2.1)
        2. [Porównanie jakości klasyfikacji](#2.2.2)
    3. [Użytkowanie](#2.3)
        1. [Przygotowanie danych](#2.3.1)
        2. [Opis input-output](#2.3.2)
3. [Contributing](#3)
4. [Źródła](#4)


<a name="1"></a>
# Wstęp
-------------------

<a name="1.1"></a>
## Cel projektu
-------------------
Projekt ma na celu przedstawienie funkcjonowania konwolucyjnych sieci neuronowych przy pomocy TensorFlow Keras oraz podstawowej bazy danych pakietu tensorflow_datasets.

<a name="1.2"></a>
## Czym jest CNN?
-------------------
Aby odpowiedzieć na pytanie, czym jest **Konwolucyjna Sieć Neuronowa** (ang. Convolutional Neural Network, inaczej CNN), pierw musimy przybliżyć pojęcie sztucznej sieci neuronowej.<br/>
Sieci neuronowe są w ogólności modelem obliczeniowym inspirowanym strukturą i funkcją ludzkiego mózgu. Składają się z dużej liczby prostych jednostek przetwarzających, zwanych sztucznymi neuronami, które są zorganizowane w warstwy i połączone ze sobą za pomocą wag i odchyleń, by jak najlepiej rozpoznać podstawowe relacje w zbiorze danych.

Rozłóżmy sieć neuronową na jej podstawowe bloki konstrukcyjne:
1. **Tensor** jest uogólnieniem macierzy na dowolną liczbę wymiarów. Są one używane do reprezentowania danych w sieci neuronowej - prawie zawsze są to dane liczbowe.
![tensor](https://drive.google.com/uc?id=1ZvNacNdu_RG0jr5U2PrwVoQsRg07Jzjb)

2. **Neuron** można przedstawić jako funkcję, która przyjmuje wiele wejść i daje jedno wyjście.
![sztuczny neuron](https://drive.google.com/uc?id=1k3OCAmfXWrhs_lv-ER75e0I9OOaUqaDQ)

3. **Warstwa** to zbiór neuronów o tym samym działaniu, z tymi samymi hiperparametrami.

4. **Wagi filtra** (inaczej wagi jądra lub maski), choć unikalne dla każdego neuronu, są dostrajane w fazie treningu w celu optymalizacji wydajności sieci na danym zadaniu.

5. **Odchylenie** (ang. bias) jest skalarem, który pozwala budować model liniowy, który nie jest ustalony na początku i ma możliwość przeszukiwania przestrzeni rozwiązań. Odchylenie jest uczone wraz z wagami filtra.
![sztuczny neuron](https://drive.google.com/uc?id=1_XR0_czVA84L2-GpoVlHluIfKWvuYN_9)

**Czym różni się CNN od klasycznego modelu uczenia maszynowego?**

CNN wykorzystuje specjalny typ warstwy - warstwę konwolucyjną, która jest dobrze przygotowana do uczenia się z obrazów i danych obrazopodobnych. Sieć taka może być wykorzystana do zadań związanych z widzeniem komputerowym, takich jak przetwarzanie obrazów, klasyfikacja, segmentacja i wykrywanie obiektów.<br/><br/>

**Czy zajmują sie poszczególne warstwy sieci?**

**Warstwa wejściowa** (ang. Input Layer) - pierwsza warstwa sieci, ma za zadanie przechwycenie sygnału i przekazanie go do kolejnych warstw sieci (zwanych warstwami ukrytymi).

**Warstwa konwolucyjna** (ang. Convolutional Layer) - stosuje filtry do danych wejściowych i uczy się rozpoznawania cech. Początkowe warstwy konwolucyjne wydobywają ogólne lub niskopoziomowe cechy, takie jak linie i krawędzie, a kolejne warstwy uczą się subtelniejszych szczegółów lub cech wysokopoziomowych. 
![warstwa konwolucyjna](https://drive.google.com/uc?id=1-qrBQ0ynVIu5X8Nr9ZaQmtFX_OKJeD6j)

**Funkcje aktywacyjne** (ang. Activation Functions) - nakładane elementarnie na dane wejściowe po nałożeniu wag. Wprowadzają one nieliniowość do sieci, pozwalając modelowi na uczenie się bardziej złożonych funkcji. Niektóre z najbardziej popularnych funkcji aktywacji obejmują:
![sztuczny neuron](https://drive.google.com/uc?id=1E23OOrObsAkbnJ27MG_pRQnDNZnB2sA6)

**Warstwa łącząca** (ang. Pooling Layer) - jest używana do zmniejszania rozmiaru objętości wejściowej, aby zmniejszyć moc obliczeniową wymaganą do przetworzenia obrazu, nie tracąc przy tym właściwości obrazu.

**Warstwa dropout** (ang. Dropout Layer) - losowo wyrzuca pewne neurony podczas treningu by zapobiec nadmiernemu dopasowaniu.

**Warstwa w pełni połączona** (ang. Fully Connected Layer albo Dense Layer) pobiera wyjście z końcowej warstwy łączącej i wykorzystują je do tworzenia predykcji przy użyciu standardowych technik sieci neuronowych. Jest zazwyczaj ostatnią warstwą przed warstwą wyjściową.

**Funkcja straty** (ang. Loss Function) - pobiera ostateczne wyjście sieci, wytworzone przez warstwę w pełni połączoną, i porównuje je z wyjściem prawdziwym. Na tej podstawie obliczany jest błąd który jest wykorzystywany do 
aktualizacji wag i odchyleń.

**Warstwa wyjściowa** (ang. Output Layer) - ostatnia warstw sieci, w której neuronach formowane są sygnały wyjściowe.<br/><br/>

**Co może doprowadzić do nadmiernego dopasowanie danych?**

**Nadmierne Dopasowanie** (ang. Overfitting) to zjawisko, które występuje, gdy model jest zbyt złożony i jest w stanie dopasować się do szumu lub losowych zmian w danych treningowych, a nie do leżących u jego podstaw wzorców.<br/>
Istnieje kilka czynników, które mogą do tego doprowadzić:
  * Niewystarczająca ilość danych

  * Duża złożoność modelu: Model o zbyt wielu parametrach lub wysokim stopniu swobody.

  * Brak regularności: Regularność jest techniką dodającą termin kary do funkcji straty, która zniechęca do dużych wag, zapobiegając przy tym dopasowaniu do szumu w danych treningowych.

  * Wyciek danych: Wyciek danych występuje, gdy informacje z zestawu testowego są używane do informowania modelu podczas szkolenia. Model będzie uzyskiwał dobre wyniki na zestawie testowym, ale nie będzie dobrze generalizować na nowe przykłady.

<a name="1.3"></a>
## Z czego składa się bee_dataset?
-------------------
[Bee_dataset](https://www.tensorflow.org/datasets/catalog/bee_dataset) 
jest zbiorem obrazów oraz odpowiadających im etykiet stworzonym do uczenia algorytmu klasyfikacji obrazów do czterech odrębnych kategorii:
  * czy pszczoła jest zainfekowana dręczem pszczelim
  * czy  pszczoła przenosi pyłek
  * czy jest to osa
  * czy pszczoła chłodzi ul machając skrzydłami

Powyższe dane zapisane są jako parametry:<br/>
```{'cooling_output': 0.0, 'pollen_output': 0.0, 'varroa_output': 0.0, 'wasps_output': 0.0}```<br/>
Jeśli dana cecha występuje przypisywane jest 1, jeśli nie 0.

![bees with labels](https://drive.google.com/uc?id=1xVVYghka5JwmhWxaQA-FBMjuvzPkadr4)

Cała baza danych składa się z 7490 przykładów i jest podzielona wyłącznie na zbiór treningowy:<br/>
```splits={'train': <SplitInfo num_examples=7490, num_shards=1>,}```<br/>
Każdy obraz ma wymiar (300, 150, 3) tzn. 300px wysokości, 150px szerokości, 3 składowe odpowiadające kolorom:<br/>
```Image(shape=(300, 150, 3), dtype=uint8)```

W bazie danych znajdują się również obrazy, które algorytm uczy się klasyfikować jako same zera (ponieważ są niepoprawne lub nie należą do żadnej kategorii) oraz obrazy spełniające więcej niżeli jedną kategorię:

![bee flawed and double](https://drive.google.com/uc?id=1-zN-SQ06V0RfPXlsdB9whXer7hoLPDn0)

Wszystkie zdjęcia zostały wykonane z góry, na zielonym tle, w tych samych odległościach pszczół od kamery. Odwrócono je tak, by głowa lub tułów znajdowały się na górze zdjęcia.<br/><br/>

**Jakiej obróbce poddaliśmy naszą bazę danych?**

Wczytujemy nasz dataset.

```
dataset = tfds.load('bee_dataset', split=['train'], as_supervised=True)
```

Ustalamy przedziały zakresów, według których będziemy dzielić nasz dataset na zbiór treningowy, walidacyjny i testowy w proporcjach:

```
splitSize = (0.0, 0.765, 0.9, 1.0)
```

Konwertujemy dataset na tablicę Numpy, a następnie dokonujemy pionowej konkatenacji i dzielimy ją według wcześniej ustalonych zakresów ```splitSize```.

```
array = np.vstack(tfds.as_numpy(dataset[0]))
ranges = list([int(array.shape[0] * x) for x in splitSize])
```

Definiujemy funkcję pomocniczą ```getDataset```, która porcjuje dataset.

```
def getDataset(ranges, idx):
    return (
        np.array(list(map(lambda x: x[0][:, :, :], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['varroa_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['pollen_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['wasps_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['cooling_output'], array[ranges[idx]: ranges[idx + 1]])))
    )
``` 

Metodą pomocniczą ```getDataset``` pobieramy porcje danych zgodne z zakresami.

```
train_images, train_labels1, train_labels2, train_labels3, train_labels4 = getDataset(ranges, 0)
validation_images, validation_labels1, validation_labels2, validation_labels3, validation_labels4 = getDataset(ranges, 1)
test_images, test_labels1, test_labels2, test_labels3, test_labels4 = getDataset(ranges, 2)
```

Wszystkie etykiety scalane są w jedną strukturę.

```
train_labels = np.dstack((train_labels1, train_labels2, train_labels3,train_labels4)).squeeze()
validation_labels = np.dstack((validation_labels1,validation_labels2, validation_labels3, validation_labels4)).squeeze()
test_labels = np.dstack((test_labels1,test_labels2,test_labels3, test_labels4)).squeeze()
```
<br/>

**Wyróżnione kroki w przetwarzaniu danych**
<br/>

  **Normalizacja** (ang. Normalisation) - najważniejszy krok w części wstępnego przetwarzania. Polega ona na przeskalowaniu wartości pikseli tak, aby mieściły się one w określonym zakresie(najczęściej 0-1). Przyspiesza proces uczenia i osłabia zależność od inicjalizacji.

  **Standaryzacja** (ang. Standardisation) - przekształcenie danych numerycznych w taki sposób, aby średnia wartości danych była równa 0, a odchylenie standardowe 1.

  **Kodowanie etykiety** (ang. Label Encoding) - przekształcenie zmiennych kategorycznych do wektorów o wartości 0 lub 1.

  **Morfologiczne przekształcenia obrazów** (ang. Morphological Image Transformations) - modyfikacje obejmujące kształt i formę obrazów. Zaliczają się do nich:
  * Tworzenie obrazu binarnego
  * Erozja (kurczenie jasnych regionów i powiększanie ciemnych)
  * Dylatacja (kurczenie ciemnych regionów i powiększanie jasnych)
  * Otwarcie (dylatacja poprzedzona erozją, proces usuwa małe jasne plamy tzw. sól)
  * Zamknięcie (erozja poprzedzona dylatacją, proces usuwa małe czarne plamy tzw. pieprz)

**Rozszerzenie danych** (ang. Data Augmentation) - zestaw technik, które sztucznie generują nowe dane z istniejących danych. Zaliczają się do nich: obrót, rotacja, losowe wycinki, zmiana koloru, dodanie szumu, utrata informacji, zmiana kontrastu.<br/><br/>


<a name="2"></a>
# Model
-------------------

<a name="2.1"></a>
## Specyfika modelu
-------------------

<a name="2.2"></a>
## Część eksperymentalna
-------------------

<a name="2.2.1"></a>
### Architektura
```

model= tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, 
                         activation="relu", 
                         input_shape=(300, 150, 3)), 
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, 
                            padding="valid"), 
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), 

  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(4, activation="sigmoid") 
])
```
![layers](https://drive.google.com/uc?id=1H5YY8okmA36AcKFIIDX5P6P2jneaeAym)

<a name="2.2.2"></a>
### Porównanie jakości klasyfikacji
-------------------
<a name="2.2.3"></a>
### Trening
Sieć była trenowana przy użyciu GPU Tesla T4, co znacznie przyspieszyło cały proces i pozwoliło osiągnąć wydajność około 41ms/step (~7s/epoch). 
```
Sat Jan 21 14:21:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    26W /  70W |      0MiB / 15109MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
```
```
Epoch 20/20
180/180 [==============================] - 7s 41ms/step - loss: 0.0058 - all: 0.9961 - varroa: 0.9884 - pollen: 0.9964 - wasps: 1.0000 - cooling: 0.9998 - val_loss: 0.2819 - val_all: 0.9453 - val_varroa: 0.8906 - val_pollen: 0.9154 - val_wasps: 0.9971 - val_cooling: 0.9779
```
-------------------
<a name="2.3"></a>
## Użytkowanie
-------------------

<a name="2.3.1"></a>
### Przygotowanie danych
-------------------
<a name="2.3.2"></a>
### Opis input-output
-------------------

<a name="3"></a>
# Contributing
-------------------

<a name="4"></a>
# Źródła
-------------------
1. [Deep Learning with Python by François Chollet, version 6, 2017](https://www.manning.com/books/deep-learning-with-python)

2. [Hands-On Machine Learning with Scikit-Learn, Keras and TensorFlow by Aurélien Géron, 2nd edition, 2019](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

3. [TensorFlow Keras Documentation](https://www.tensorflow.org/api_docs/python/tf/keras)

4. [CNN Explainer](https://github.com/poloclub/cnn-explainer)

5. [Deep Learning cheatsheets for Stanford's CS 230](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-deep-learning-tips-and-tricks)

6. [Jaki jest związek pomiędzy sieciami CNN i głębokim uczeniem?](https://www.intel.pl/content/www/pl/pl/internet-of-things/computer-vision/convolutional-neural-networks.html)

7. Wykład "Uczenie maszynowe w Pythonie" dr Macieja Ślęczki

8. [Wikipedia](http://wikipedia.org)


<br/>

![zdjecie](https://drive.google.com/uc?id=16xYDkCsPLDC8VhVVwB5bq3rpxdC-fnYM)


<style type="text/css">
    ol { list-style-type: decimal; }
</style>


- contributing
- użytkowanie